In [1]:
import torch
from torchvision import (
    datasets,
    transforms
)
from torch.utils import data

import matplotlib.pyplot as plt
import seaborn as sns

plt.style.use("seaborn-v0_8")
plt.rcParams["font.family"] = "monospace"

In [2]:
if torch.cuda.is_available():
    print("CUDA found, bounding...")
    DEVICE = torch.device('cuda:0')
else:
    print("NO CUDA found, bounding to CPU...")
    DEVICE = torch.device('cpu')

CUDA found, bounding...


In [3]:
torch.cuda.get_device_properties(DEVICE)

_CudaDeviceProperties(name='NVIDIA GeForce RTX 4060 Laptop GPU', major=8, minor=9, total_memory=8187MB, multi_processor_count=24)

In [4]:
DTYPE = torch.float32

Arithmetic Operations
-> Elementwise Operations => Between two elements of two tensors (index/location matters)
* ADDING, SUBTRACTING, MULTIPLYING, DIVIDING
( ! ) Same sized/Shaped Tensors

In [10]:
A = torch.rand((5, 5), dtype=DTYPE, device=DEVICE)
A, A.shape

(tensor([[0.6055, 0.4606, 0.7555, 0.7673, 0.8293],
         [0.2140, 0.7832, 0.1364, 0.5822, 0.7886],
         [0.7660, 0.5519, 0.3378, 0.4343, 0.8945],
         [0.9205, 0.4017, 0.4399, 0.9196, 0.9024],
         [0.2814, 0.2968, 0.9291, 0.7122, 0.7486]], device='cuda:0'),
 torch.Size([5, 5]))

In [11]:
B = torch.rand((5, 5), dtype=DTYPE, device=DEVICE)
B, B.shape

(tensor([[0.6139, 0.1199, 0.9283, 0.1613, 0.4278],
         [0.7453, 0.8040, 0.2216, 0.6554, 0.6120],
         [0.9504, 0.3979, 0.0540, 0.3131, 0.1012],
         [0.4855, 0.4938, 0.4480, 0.8944, 0.9936],
         [0.7357, 0.1531, 0.1091, 0.3037, 0.1201]], device='cuda:0'),
 torch.Size([5, 5]))

In [13]:
.6055 + .6139

1.2194

In [12]:
#-Addıng
AB = torch.add(A, B)
AB, AB.shape

(tensor([[1.2195, 0.5805, 1.6838, 0.9286, 1.2571],
         [0.9593, 1.5871, 0.3580, 1.2375, 1.4006],
         [1.7164, 0.9498, 0.3918, 0.7474, 0.9957],
         [1.4061, 0.8955, 0.8878, 1.8141, 1.8960],
         [1.0172, 0.4499, 1.0382, 1.0159, 0.8688]], device='cuda:0'),
 torch.Size([5, 5]))

In [16]:
torch.add(A, B).all() == (A + B).all()

tensor(True, device='cuda:0')

In [18]:
.6055 - .6139

-0.008399999999999963

In [17]:
# Subtracting
AB = torch.sub(A, B)
AB, AB.shape

(tensor([[-0.0084,  0.3407, -0.1728,  0.6060,  0.4015],
         [-0.5313, -0.0208, -0.0851, -0.0732,  0.1766],
         [-0.1844,  0.1540,  0.2837,  0.1213,  0.7934],
         [ 0.4350, -0.0922, -0.0081,  0.0252, -0.0912],
         [-0.4543,  0.1437,  0.8200,  0.4085,  0.6285]], device='cuda:0'),
 torch.Size([5, 5]))

In [19]:
AB = A - B
AB

tensor([[-0.0084,  0.3407, -0.1728,  0.6060,  0.4015],
        [-0.5313, -0.0208, -0.0851, -0.0732,  0.1766],
        [-0.1844,  0.1540,  0.2837,  0.1213,  0.7934],
        [ 0.4350, -0.0922, -0.0081,  0.0252, -0.0912],
        [-0.4543,  0.1437,  0.8200,  0.4085,  0.6285]], device='cuda:0')

In [20]:
torch.sub(A, B).all() == (A - B).all()

tensor(True, device='cuda:0')

In [23]:
# Multiplying
AB = torch.mul(A, B)
AB

tensor([[0.3718, 0.0552, 0.7013, 0.1237, 0.3548],
        [0.1595, 0.6296, 0.0302, 0.3815, 0.4826],
        [0.7280, 0.2196, 0.0182, 0.1360, 0.0905],
        [0.4469, 0.1984, 0.1970, 0.8226, 0.8966],
        [0.2071, 0.0454, 0.1014, 0.2163, 0.0899]], device='cuda:0')

In [22]:
A * B

tensor([[0.3718, 0.0552, 0.7013, 0.1237, 0.3548],
        [0.1595, 0.6296, 0.0302, 0.3815, 0.4826],
        [0.7280, 0.2196, 0.0182, 0.1360, 0.0905],
        [0.4469, 0.1984, 0.1970, 0.8226, 0.8966],
        [0.2071, 0.0454, 0.1014, 0.2163, 0.0899]], device='cuda:0')

In [24]:
.6055 * .6139

0.37171645000000003

In [26]:
# Dividing
AB = torch.div(A, B)
AB

tensor([[0.9863, 3.8408, 0.8139, 4.7580, 1.9384],
        [0.2872, 0.9741, 0.6157, 0.8883, 1.2885],
        [0.8060, 1.3870, 6.2517, 1.3873, 8.8431],
        [1.8960, 0.8133, 0.9819, 1.0282, 0.9082],
        [0.3825, 1.9387, 8.5129, 2.3449, 6.2325]], device='cuda:0')

In [27]:
.6055 / .6139

0.9863169897377424

In [28]:
A / B

tensor([[0.9863, 3.8408, 0.8139, 4.7580, 1.9384],
        [0.2872, 0.9741, 0.6157, 0.8883, 1.2885],
        [0.8060, 1.3870, 6.2517, 1.3873, 8.8431],
        [1.8960, 0.8133, 0.9819, 1.0282, 0.9082],
        [0.3825, 1.9387, 8.5129, 2.3449, 6.2325]], device='cuda:0')

BROADCASTING
-> Same SHAPE Tensors
* What if they are not the same?
Vector - Matrix (1D - 2D tensors)

In [33]:
v = torch.tensor([1, 3, 0, 5, 10], dtype=DTYPE, device=DEVICE)
v, v.shape, v.ndim

(tensor([ 1.,  3.,  0.,  5., 10.], device='cuda:0'), torch.Size([5]), 1)

In [35]:
M = torch.tensor([
    [1, 1, 2, 2, 3],
    [0, 0, 5, 4, 0],
    [10, 0, 1, 1, 5]], dtype=DTYPE, device=DEVICE)
M, M.shape, M.ndim

(tensor([[ 1.,  1.,  2.,  2.,  3.],
         [ 0.,  0.,  5.,  4.,  0.],
         [10.,  0.,  1.,  1.,  5.]], device='cuda:0'),
 torch.Size([3, 5]),
 2)

In [42]:
Mv = torch.mul(M, v)
Mv

tensor([[ 1.,  3.,  0., 10., 30.],
        [ 0.,  0.,  0., 20.,  0.],
        [10.,  0.,  0.,  5., 50.]], device='cuda:0')

Broadcasting for 1D - 2D Tensors

v = [ 1.,  3.,  0.,  5., 10.]

M = [ 1.,  1.,  2.,  2.,  3.],
    [ 0.,  0.,  5.,  4.,  0.],
    [10.,  0.,  1.,  1.,  5.]
    
v -> V = [ 1.,  3.,  0.,  5., 10.]
         [ 1.,  3.,  0.,  5., 10.]
         [ 1.,  3.,  0.,  5., 10.]

[1., 3., 0., 10., 30.],
[0., 0., 0., 20., 0.],
[10., 0., 0., 5., 50.]

TRANSPOSE
* A(M, N), b(N)
* A.T ==> (N, M)
Transpose: COLUMNS -> rows, ROWS -> columns
[1, 2, 3, 4, 5, 6, ... N] -> (1, N) ==> (N, 1)

Transpose of a Vector : COLUMN vector
[[1.],
[2.],
[3.],
[4.],
[5.],
[6.],
.
.
.
[N.]]

In [44]:
M, M.shape

(tensor([[ 1.,  1.,  2.,  2.,  3.],
         [ 0.,  0.,  5.,  4.,  0.],
         [10.,  0.,  1.,  1.,  5.]], device='cuda:0'),
 torch.Size([3, 5]))

In [45]:
MT = M.T
MT, MT.shape

(tensor([[ 1.,  0., 10.],
         [ 1.,  0.,  0.],
         [ 2.,  5.,  1.],
         [ 2.,  4.,  1.],
         [ 3.,  0.,  5.]], device='cuda:0'),
 torch.Size([5, 3]))

In [48]:
v = torch.tensor([[1, 3, 0, 5, 10]], dtype=DTYPE, device=DEVICE)
v, v.shape, v.ndim

(tensor([[ 1.,  3.,  0.,  5., 10.]], device='cuda:0'), torch.Size([1, 5]), 2)

In [49]:
vT = v.T
vT, vT.shape

(tensor([[ 1.],
         [ 3.],
         [ 0.],
         [ 5.],
         [10.]], device='cuda:0'),
 torch.Size([5, 1]))